In [ ]:
library(devtools)
library(denovolyzeR)
library(reshape2)
library(dplyr)
options(stringsAsFactors = F)

In [2]:
###########
# Setting #
###########
today <- format(Sys.Date(), "%Y%m%d")

In [3]:
#############
# FUNCTIONS #
#############
# Check the existence of file or directory
check_path_files <- function(path) {
    if (dir.exists(path)) {
        full_path <- normalizePath(path, winslash = "/")  # Get full path in a consistent format
        return(paste(full_path, "- This is a valid directory."))
        } else if (file.exists(path)) {
        file_name <- basename(path)  # Extract the file name from the path
        return(paste(file_name, "- This is a valid file."))
        } else {
        return("The path does not exist.")
        }
}

# Create the directory if doesn't exist
create_folder_if_not_exists <- function(path) {
  if (!dir.exists(path)) {  # Use ! to negate the condition
    dir.create(path, recursive = TRUE)  # Create directory if it doesn't exist
    full_path <- normalizePath(path, winslash = "/")  # Get the full path after creation
    return(paste(full_path, "- Directory was created."))
  } else {
    full_path <- normalizePath(path, winslash = "/")  # Get full path if directory exists
    return(paste(full_path, "- Directory already exists."))
  }
}

# Reformat the mutablity table
reformat_pDNM <- function(x, mis_filter="Mis_MetaSVM_or_CADD20"){  # MetaSVM or CADD20
  names(x)[names(x)==mis_filter] <- "misD"
  x$prot <- x[,names(x) %in% c("lof","mis","misD")] %>% apply(MARGIN=1,sum)
  x$protD <- x[,names(x) %in% c("lof","misD")] %>% apply(MARGIN=1,sum)
  x <- melt(x)                                          #use anything that is 'chr' as id variables
  names(x)[names(x)=="variable"] <- "class"             #change whichever column with the name 'variable' to 'class'
  return(x)
}

# Gene level table annotation
calculate_pvalues <- function(data, gnomad_file, mutability_table) {
    # Step 1: Read the Gnomad data file with pLI and misZ columns
    gnomad_data <- read.delim(gnomad_file, header = TRUE, stringsAsFactors = FALSE)
    
    # Step 2: Merge input data with Gnomad data based on gene names
    merged_data <- merge(data, gnomad_data, by.x = "gene", by.y = "MutTable_Hydro_GeneName", all.x = TRUE)
    
    # Step 3: Automatically find columns containing "_pValue"
    pvalue_columns <- grep("_pValue", names(merged_data))
    
    # Step 4: Calculate the minimum value for each row across the found "_pValue" columns
    merged_data$min_p <- apply(merged_data[, pvalue_columns], 1, min, na.rm = TRUE)
    
    # Step 5: Return the updated data frame
    return(merged_data)
}

# Check if the gene in the input exists in the mutablity table
find_excluded_genes <- function(input_genes, prob_table_genes) {
  # Step 1: Identify genes in input_genes that are not in prob_table_genes
  excluded_genes <- setdiff(input_genes, prob_table_genes)
  
  # Step 2: Print the excluded genes or a message if none are excluded
  if (length(excluded_genes) > 0) {
    cat("Excluded genes:\n")
    print(excluded_genes)
  } else {
    cat("No genes were excluded.\n")
  }
  
  # Return the list of excluded genes
  return(excluded_genes)
}

# Reformat the enrichment result table
reformat_class_enrichment <- function(data, case_number) {
  
  # Step 1: Rename specific columns
  data$class <- as.character(data$class)
  data$class[data$class == "all"] <- "Total"
  data$class[data$class == "syn"] <- "Synonymous"
  data$class[data$class == "misD"] <- "D-Mis"
  data$class[data$class == "lof"] <- "LoF"
  data$class[data$class == "prot"] <- "Protein-altering"
  data$class[data$class == "protD"] <- "Damaging"
  
  # Step 2: Remove the 'mis' row, but calculate the "T-Mis" row
  mis_row <- data[data$class == "mis", ]
  misD_row <- data[data$class == "D-Mis", ]
  
  observed_n <- mis_row$observed - misD_row$observed
  expected_n <- mis_row$expected - misD_row$expected
  enrichment <- observed_n / expected_n
  pvalue <- ppois(observed_n - 1, lambda = expected_n, lower.tail = FALSE)
  
  # Step 3: Calculate Observed Rate and Expected Rate for "T-Mis"
  observed_rate <- observed_n / case_number
  expected_rate <- expected_n / case_number
  
  # Create the "T-Mis" row
  tmis_row <- data.frame(
    class = "T-Mis",
    observed = observed_n,
    observed_rate = observed_rate,
    expected = expected_n,
    expected_rate = expected_rate,
    enrichment = enrichment,
    pValue = pvalue
  )
  
  # Step 4: Calculate Observed Rate and Expected Rate for existing rows
  data$observed_rate <- data$observed / case_number
  data$expected_rate <- data$expected / case_number
  
  # Step 5: Remove 'mis' row and add 'T-Mis' row
  data <- data[data$class != "mis", ]
  data <- rbind(data, tmis_row)
  
  # Step 6: Reorder rows to match the desired order
  row_order <- c("Total", "Synonymous", "T-Mis", "D-Mis", "LoF", "Protein-altering", "Damaging")
  data <- data[match(row_order, data$class), ]
  
  # Step 7: Reorder columns to match the desired order
  data <- data[, c("class", "observed", "observed_rate", "expected", "expected_rate", "enrichment", "pValue")]
  
  # Step 8: Return the reformatted data
  return(data)
}


In [4]:
####################
# Cohort Trio Size #
####################
case_num = 141 # <- modified for each analysis

In [ ]:
#####################
# working directory #
#####################
working_directory = "Denovo_enrichment"
check_path_files(working_directory)

In [ ]:
#########
# Files #
#########
input_file = file.path(working_directory, "Input", "2023-WES_CP_CONG_Denovo_enrichment_input_161.txt")
check_path_files(input_file)

lof_gene_list = file.path(working_directory, "Datasets", "LoF-Intolerant-Genes-gnomAD2.1.1.txt")
check_path_files(lof_gene_list)

mutability_table = file.path(working_directory, "Datasets", "20240927-Exome-IDT_V1V2_span50bp_removecolumn.txt")
check_path_files(mutability_table)

gnomad_file = file.path(working_directory, "Datasets", "Gnomadv2-pLI_misZ-by_gene.txt")
check_path_files(gnomad_file)

In [ ]:
##########
# Output #
##########
out_folder_name = paste0(today,"-Denovo_enrichment_anaysis_Results") # <- modified for each analysis
out_directory = file.path(working_directory, out_folder_name)
create_folder_if_not_exists(out_directory)

gene_level_file = file.path(out_directory, "Gene_Level_Significance_MetaSVM_or_CADD20.txt")
class_enrichment_file = file.path(out_directory, "Class_enrichment_gene_all.txt")
lof_enrichment_file = file.path(out_directory, "Class_enrichment_gene_lof.txt")
qq_plot_gene_level_file = file.path(out_directory, "qqplot_All-Gene_Level_Significance_MetaSVM_or_CADD20.jpeg")

### Start test here

In [9]:
#################
# Read-in Files #
#################
## Read-in Denovo list (different by cohort)
raw_case <- read.table(file = input_file, header = TRUE, stringsAsFactors = FALSE, sep = "\t")

## Read-in Lof gene list (preset)
Int_genes <- read.table(file = lof_gene_list, header = TRUE, stringsAsFactors = FALSE, sep = "\t")

## Read-in Mut table
cases10 <- read.delim(file = mutability_table, header = TRUE, stringsAsFactors = FALSE, sep = "\t")


In [ ]:
#################
## Denovo list ##
#################
raw_case$Cases10GeneName <- toupper(raw_case$Cases10GeneName)

## Set cases
cases = raw_case

## Remove the denovo variants at same gene in same sample (column: Duplicate)
dup <- toupper(cases$Duplicate)
index <- which(dup=="YES")
if(length(index) !=0){
    cases <- cases[-index,]
}

## Check the lengths of genes and classes
length_genes <- length(cases$Cases10GeneName)
length_classes <- length(cases$Mis_MetaSVM_or_CADD20)

## Compare the lengths
if (length_genes != length_classes) {
  stop("The number of genes and number of variant consequences do not match.")
} else {
  print("Length matched.")
}

In [11]:
###################
## Lof Gene List ##
###################
Int_genes <- toupper(Int_genes$gene_name_mut_table)

## remove name is N/A
index <- which(Int_genes=="N/A")
if(length(index) != 0){
	Int_genes <- Int_genes[-index]
}

In [ ]:
######################
## Mutability Table ##
######################
## Reformatting the mutability table
unique(reformat_pDNM(cases10)[,"class"])
pDNM_cases10 <- reformat_pDNM(cases10)
pDNM_cases10$gene <- toupper(pDNM_cases10$gene)

In [ ]:
###########################################################
# Check if Input genes are all appear in Mutability Table #
###########################################################
# Step 1: Extract gene names from cases and probability table
input_genes <- unique(cases$Cases10GeneName)  # Unique genes in the input list
prob_table_genes <- unique(pDNM_cases10$gene)  # Assuming the gene column in pDNM_cases10 is named 'gene'
find_excluded_genes(input_genes, prob_table_genes)

In [14]:
#########################
## Enrichment analysis ##
#########################
## Class enrichment analysis (All gene)
class_enrichment <- denovolyzeByClass(genes=cases$Cases10GeneName,classes= cases$Mis_MetaSVM_or_CADD20,geneId="gene", nsamples=case_num, probTable=pDNM_cases10, includeClasses=c("all", "syn", "mis", "protD", "misD","lof", "prot"))
class_enrichment <- reformat_class_enrichment(class_enrichment, case_num)
class_enrichment
write.table(class_enrichment, file = class_enrichment_file, col.names = TRUE, row.names = FALSE, sep = "\t", append = FALSE, quote = FALSE)

## Class enrichment analysis (Lof gene)
class_lof_enrichment <- denovolyzeByClass(genes=cases$Cases10GeneName,classes= cases$Mis_MetaSVM_or_CADD20,geneId="gene", nsamples=case_num, probTable=pDNM_cases10, includeGenes=Int_genes, includeClasses=c("all", "syn", "mis", "protD", "misD","lof", "prot"))
class_lof_enrichment <- reformat_class_enrichment(class_lof_enrichment, case_num)
class_lof_enrichment
write.table(class_lof_enrichment, file = lof_enrichment_file, col.names = TRUE, row.names = FALSE, sep = "\t", append = FALSE, quote = FALSE)

## Gene level significance
casesByGene <- denovolyzeByGene(cases$Cases10GeneName,cases$Mis_MetaSVM_or_CADD20, case_num, geneId="gene",probTable=pDNM_cases10, includeGenes="all", signifP=3, roundExpected =15, includeClasses=c("protD", "lof", "misD","prot"))
casesByGene <- cbind.data.frame(rownames(casesByGene),casesByGene)
colnames(casesByGene)[1] = 'GeneOrder'
casesByGene <- calculate_pvalues(casesByGene, gnomad_file, mutability_table)
casesByGene
write.table(casesByGene, file = gene_level_file, col.names = TRUE, row.names = FALSE, sep = "\t", append = FALSE, quote = FALSE)

## Generate qq plot from gene level signigicance
number_denovo_genes <- nrow(casesByGene)
number_mutability_table_genes <- nrow(cases10)

pvalue <- append(casesByGene$min_p,rep(1, number_mutability_table_genes - number_denovo_genes))

observed <- sort(as.numeric(pvalue), decreasing = FALSE)
log_obs <- -log10(observed)

expected <- c(1:length(observed))
log_exp <- -(log10(expected / (length(expected)+1)))

jpeg(qq_plot_gene_level_file, width = 6, height = 6, units = "in", res = 600) # Save plot as JPEG with resolution 600 DPI

### Create the plot
plot(c(0, 16), c(0, 16), col = "red", lwd = 3, type = "l", 
     xlab = "Expected (-logP)", ylab = "Observed (-logP)", 
     xlim = c(0, 16), ylim = c(0, 16), las = 1, 
     xaxs = "i", yaxs = "i", bty = "l")
points(log_exp, log_obs, pch = 23, cex = 0.4, bg = "black")

### Close the JPEG device
dev.off()

,class,observed,observed_rate,expected,expected_rate,enrichment,pValue
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7,Total,160,1.1347518,176.5,1.2517730,0.9070000,0.9010000
1,Synonymous,30,0.2127660,49.9,0.3539007,0.6010000,0.9990000
11,T-Mis,24,0.1702128,39.9,0.2829787,0.6015038,0.9973265
2,D-Mis,85,0.6028369,71.4,0.5063830,1.1900000,0.0633000
4,LoF,21,0.1489362,15.2,0.1078014,1.3800000,0.0936000
5,Protein-altering,130,0.9219858,197.9,1.4035461,0.6570000,1.0000000
6,Damaging,106,0.7517730,86.6,0.6141844,1.2200000,0.0240000


Warning message in parseInput(genes, classes, nsamples, groupBy, includeGenes, includeClasses, :
“45 gene identifiers in "includeGene" are not in the probability table, and are excluded from analysis.”


,class,observed,observed_rate,expected,expected_rate,enrichment,pValue
,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7,Total,40,0.28368794,41.6,0.29503546,0.9630000,0.6160000
1,Synonymous,7,0.04964539,11.7,0.08297872,0.5980000,0.9460000
11,T-Mis,3,0.02127660,6.9,0.04893617,0.4347826,0.9680482
2,D-Mis,22,0.15602837,19.3,0.13687943,1.1400000,0.3020000
4,LoF,8,0.05673759,3.7,0.02624113,2.1700000,0.0348000
5,Protein-altering,33,0.23404255,49.2,0.34893617,0.6710000,0.9940000
6,Damaging,30,0.21276596,23.0,0.16312057,1.3000000,0.0929000


gene,GeneOrder,misD_observed,misD_expected,misD_pValue,lof_observed,lof_expected,lof_pValue,prot_observed,prot_expected,prot_pValue,protD_observed,protD_expected,protD_pValue,pLI,mis_z,min_p
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
ACSL3,1,1,0.00448380,0.00447,0,0.001026480,1.000000,1,0.011122080,0.01110,1,0.005510280,0.00550,3.48e-04,2.0100,0.004470
ACTR5,2,0,0.00552720,1.00000,1,0.000679620,0.000679,1,0.012580020,0.01250,1,0.006206820,0.00619,1.14e-09,0.4050,0.000679
ADAD2,3,0,0.00355320,1.00000,0,0.000648600,1.000000,0,0.011674800,1.00000,0,0.004201800,1.00000,2.39e-12,-2.3600,1.000000
ADAM15,4,0,0.00549900,1.00000,0,0.001570740,1.000000,1,0.015698940,0.01560,0,0.007069740,1.00000,2.44e-16,1.0700,0.015600
ADAMTS9,5,1,0.01285920,0.01280,0,0.003327600,1.000000,1,0.033981000,0.03340,1,0.016186800,0.01610,1.00e-06,0.9550,0.012800
ADD1,6,0,0.00597840,1.00000,0,0.000919320,1.000000,1,0.014229720,0.01410,0,0.006897720,1.00000,9.88e-01,0.8040,0.014100
AFF3,7,1,0.00747300,0.00745,0,0.001542540,1.000000,1,0.021113340,0.02090,1,0.009015540,0.00898,1.00e+00,1.7100,0.007450
AGTPBP1,8,1,0.00640140,0.00638,0,0.001849920,1.000000,1,0.018459720,0.01830,1,0.008251320,0.00822,4.22e-01,2.0100,0.006380
ANGEL2,9,1,0.00269310,0.00269,0,0.000919320,1.000000,1,0.007588620,0.00756,1,0.003612420,0.00361,5.57e-06,0.8820,0.002690


png 
  2

In [ ]:
number_mutability_table_genes

## Others

In [ ]:
View(denovolyze)

In [ ]:
ppois

In [ ]:
##################################
## Calculate pvalue using ppois ##
##################################
ob <- 3
exp <- 4.7
pvalue <- ppois(ob -1, lambda=exp, lower.tail=F)
pvalue